# 0. Load data

In [294]:

import tensorflow_hub as hub
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql import types as t
from pyspark.sql import Window as w

from pyspark.ml.linalg import DenseVector, SparseVector
from pyspark.ml.feature import HashingTF, IDF,  Tokenizer, RegexTokenizer, CountVectorizer, StopWordsRemover, NGram, Normalizer, VectorAssembler, Word2Vec, Word2VecModel, PCA
from pyspark.ml import Pipeline, Transformer
from pyspark.ml.clustering import LDA
from pyspark.mllib.linalg import Vectors, VectorUDT

spark = SparkSession.builder.getOrCreate()

In [295]:
df = spark.read.csv("data/X2.csv", header=True)
df.toPandas()

,instance_id,brand,cpu_brand,cpu_model,cpu_type,cpu_frequency,ram_capacity,ram_type,ram_frequency,hdd_capacity,ssd_capacity,weight,dimensions,title
0,www.softwarecity.ca//737,Lenovo,Intel. i5-3320M,i5-3320M,Dual-core ( 2 Core ). Core i5,2.60 GHz,None,DDR3 SDRAM. DDR3-1600/PC3-12800. DDR3 SDRAM,DDR3-1600/PC3-12800,320 GB,None,1.80 kg,None,"""Lenovo Thinkpad X230 34352jf Tablet Pc - 12.5..."
1,www.isupplyhub.com//1256,Acer,1.6 GHz Intel Core i5-4200U. Intel Core I5,None,1.6 GHz Intel Core i5-4200U,1.6 GHz Intel Core i5-4200U,8 GB DDR3L SDRAM,DDR3 SDRAM. 8 GB DDR3L SDRAM,None,500 GB mechanical_hard_drive,None,4.8 pounds,15.02 x 10.08 x 0.90 inches,Amazon.com : Acer Aspire V7-582PG-6479 15.6-In...
2,www.isupplyhub.com//326,Acer,1.6 GHz Intel Core i5. Intel Core I5,None,1.6 GHz Intel Core i5,1.6 GHz Intel Core i5,4 GB DDR3-SDRAM,DDR3 SDRAM. 4 GB DDR3-SDRAM,None,500 GB mechanical_hard_drive,None,5.2 pounds,15.02 x 10.08 x 1 inches,Amazon.com : Acer Aspire E1-572-6870 15.6 Inch...
3,www.isupplyhub.com//821,HP,None,None,None,None,4 GB SDRAM DDR3,DDR3 SDRAM. 4 GB SDRAM DDR3,None,500 GB,None,4.8 pounds,15.18 x 0.89 x 10.16 inches,"""Amazon.com : 15.6"""" HP 15-f009wm Amd Dual-Cor..."
4,www.isupplyhub.com//157,Asus,1.7 GHz Core i5-3317U. Intel,None,1.7 GHz Core i5-3317U,1.7 GHz Core i5-3317U,4 GB DDR3,DDR3 SDRAM. 4 GB DDR3,None,256 MB,None,2.9 pounds,8.80 x 0.70 x 12.80 inches,Amazon.com : ASUS UX31A-XB52 13.3-Inch Ultrabo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,www.vology.com//873,Lenovo ThinkPad X230 2320 - 12.5 '' - Core i5 ...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,4 GB DDR3 Slots Qty 2 Empty Slots 1 Max RAM Su...,4 GB DDR3 Slots Qty 2 Empty Slots 1 Max RAM Su...,4 GB DDR3 Slots Qty 2 Empty Slots 1 Max RAM Su...,180 GB SSD. 180 GB SSD. Lenovo ThinkPad X230 2...,180 GB SSD. 180 GB SSD,3.3 lbs 3.3 lbs,8.1 in. 12 in x 8.1 in x 1 in. 1 in. 12 in,"""Lenovo ThinkPad X230 2320 - 12.5"""" - Core i5 ..."
339,www.vology.com//823,Lenovo ThinkPad X230 2325 - 12.5 '' - Core i5 ...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,4 GB DDR3 Slots Qty 2 Max RAM Supported 16 GB ...,4 GB DDR3 Slots Qty 2 Max RAM Supported 16 GB ...,4 GB DDR3 Slots Qty 2 Max RAM Supported 16 GB ...,500 GB HDD / 7200 rpm. 500 GB HDD / 7200 rpm. ...,500 GB HDD / 7200 rpm. 500 GB HDD / 7200 rpm,3.3 lbs 3.3 lbs,8.1 in. 12 in x 8.1 in x 1 in. 1 in. 12 in,"""Lenovo ThinkPad X230 2325 - 12.5"""" - Core i5 ..."
340,www.vology.com//2723,Lenovo ThinkPad X230 Tablet 3438 - 12.5 '' - C...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Form Factor SO DIMM 204-pin Technology DDR3 SD...,Form Factor SO DIMM 204-pin Technology DDR3 SD...,Form Factor SO DIMM 204-pin Technology DDR3 SD...,500 GB HDD / 7200 rpm. 500 GB HDD / 7200 rpm. ...,500 GB HDD / 7200 rpm. 500 GB HDD / 7200 rpm,4 lbs 4 lbs,9 in. 12 in x 9 in x 1.2 in. 1.2 in. 12 in,"""Lenovo ThinkPad X230 Tablet 3438 - 12.5"""" - C..."
341,www.vology.com//1349,Lenovo ThinkPad X230 2324 - 12.5 '' - Core i5 ...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Form Factor SO DIMM 204-pin Technology DDR3 SD...,Form Factor SO DIMM 204-pin Technology DDR3 SD...,Form Factor SO DIMM 204-pin Technology DDR3 SD...,320 GB HDD / 7200 rpm. 320 GB HDD / 7200 rpm. ...,320 GB HDD / 7200 rpm. 320 GB HDD / 7200 rpm,3.3 lbs 3.3 lbs,8.1 in. 12 in x 8.1 in x 1 in. 1 in. 12 in,"""Lenovo ThinkPad X230 2324 - 12.5"""" - Core i

# 0. Data cleaning

In [296]:
for c in df.columns:
#set everything to lowercase
    df = df.withColumn(c, f.lower(f.col(c)))

#extract brand or infer from title
df = df.withColumn('brand', f.regexp_extract('brand', "^(\w+)", 1))
computer_brands = ['(lenovo', 'acer', 'hp', 'dell', 'asus', 'samsung', 'huawei', 'surface', 'apple)']
computer_brands_pattern = '|'.join(computer_brands)
df = df.withColumn('brand', f.when( f.regexp_extract('title', computer_brands_pattern, 1)!='', f.regexp_extract('title', computer_brands_pattern, 1))\
                   .otherwise(df.brand))
df.select('brand').show()

#exctract cpu_brand and infer type if intel
cpu_brands = ['(intel', 'apple', 'amd', 'nvidia', 'arm)']
cpu_pattern = '|'.join(cpu_brands)
df = df.withColumn('cpu_brand', f.when(f.regexp_extract('cpu_brand', cpu_pattern, 1) != '', f.regexp_extract('cpu_brand', cpu_pattern, 1))\
                                       .otherwise(f.regexp_extract('title', cpu_pattern, 1)))
df = df.withColumn('cpu_model', f.when(df.cpu_brand == 'intel', f.regexp_extract('cpu_model', '((i\d)|pentium|celeron)', 1))\
                   .otherwise(df.cpu_model))
df.select('cpu_model').show()

+------+
| brand|
+------+
|lenovo|
|  acer|
|  acer|
|    hp|
|  asus|
|lenovo|
|  acer|
|  acer|
|lenovo|
|  acer|
|  dell|
|  dell|
|  dell|
|  acer|
|  acer|
|  acer|
|lenovo|
|  dell|
|  acer|
|  acer|
+------+
only showing top 20 rows

+---------+
|cpu_model|
+---------+
|       i5|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
+---------+
only showing top 20 rows



# 1. Blocking

In [297]:
"""UTILITIES"""

"""Returns the df with tokenized columns with stopwords removed"""
def tokenize(df, string_cols):
  output = df
  stopW = ['amazon', 'com' , 'computer', 'accessories', 'laptop', 'notebook', 'gb', 'ram', 'hdd', 'sdd', 'cpu', 'hz', 'ghz', 'tb', 'cache', 'ddram', 'dram']
  for c in string_cols:
    output = output.withColumn('temp', f.coalesce(f.col(c), f.lower(c), f.lit('')))
    output.select(c).show()
    tokenizer = RegexTokenizer(inputCol='temp', outputCol=c+"_tokens", pattern = "\\W")
    remover = StopWordsRemover(inputCol=c+"_tokens", outputCol=c+"_swRemoved", stopWords=stopW)
    output = tokenizer.transform(output)
    output = remover.transform(output)\
      .drop('temp', c+"_tokens")
    output.show()
    # output has c+swRemoved columns
  return output

"""Selects the n top keywords from the tokens in each row from a vocab(from one col) using TFIDF score"""
def top_kw_from_tfidf(vocab, n=3):
  @f.udf(returnType=t.ArrayType(t.StringType()))
  def _(arr):
    inds = arr.indices
    vals = arr.values
    top_inds = vals.argsort()[-n:][::-1]
    top_keys = inds[top_inds]
    output = []

    for k in top_keys:
      kw = vocab.value[k]
      output.append(kw)

    return output
  return _

"""Selects top n tokens using TFIDF score for each token_cols using above func"""
def tfidf_top_tokens(df, token_cols, min_freq=1):
  for c in token_cols:

    cv = CountVectorizer(inputCol=c, outputCol=c+"_RawFeatures")
    cvmodel = cv.fit(df)
    df_vect = cvmodel.transform(df)

    idf = IDF(inputCol=c+"_RawFeatures", outputCol=c+"_features", minDocFreq=min_freq)
    idfModel = idf.fit(df_vect)
    df_idf= idfModel.transform(df_vect)

    normalizer = Normalizer(p=2.0, inputCol=c+'_features', outputCol=c+'_tfidf')
    output = normalizer.transform(df_idf)

    corpus = output.select(c+'_tfidf').rdd.zipWithIndex()
    corpus.foreach(lambda t : (t[1],t[0])) #

    lda = LDA(k=10, maxIter=100, featuresCol=c+'_tfidf')
    lda_model = lda.fit(output)
    vocab = cvmodel.vocabulary

    def get_words(token_list):
        return [vocab[token_id] for token_id in token_list]

    udf_to_words = f.udf(get_words, t.ArrayType(t.StringType()))

    topics = lda_model.describeTopics(5).withColumn('topicWords', udf_to_words(f.col('termIndices')))\
        .show()

  return output

"""Use universal sentence encoder from tensorflow_hub"""
MODEL = None
def get_model_magic():
  global MODEL
  if MODEL is None:
      MODEL = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
  return MODEL

@f.udf(returnType=VectorUDT())
def encode_sentence(x):
  model = get_model_magic()
  emb = model([x]).numpy()[0]
  return Vectors.dense(emb)

In [298]:
columns = ['title', 'brand', 'cpu_brand']
blocking_df = tokenize(df, columns)
#use N-Gram

+--------------------+
|               title|
+--------------------+
|"lenovo thinkpad ...|
|amazon.com : acer...|
|amazon.com : acer...|
|"amazon.com : 15....|
|amazon.com : asus...|
|amazon.com : leno...|
|amazon.com : acer...|
|amazon.com : acer...|
|amazon.com : leno...|
|amazon.com : acer...|
|amazon.com : dell...|
|amazon.com : dell...|
|amazon.com : dell...|
|amazon.com : acer...|
|amazon.com : acer...|
|amazon.com : acer...|
|"amazon.com : len...|
|amazon.com : dell...|
|amazon.com : acer...|
|amazon.com : acer...|
+--------------------+
only showing top 20 rows

+--------------------+------+---------+---------+--------------------+--------------------+-----------------+--------------------+-------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+
|         instance_id| brand|cpu_brand|cpu_model|            cpu_type|       cpu_frequency|     ram_capacity|            ram_type|      ram_frequency|   

In [299]:
blocking_df = tfidf_top_tokens(blocking_df, [c+'_swRemoved' for c in columns])


+-----+--------------------+--------------------+--------------------+
|topic|         termIndices|         termWeights|          topicWords|
+-----+--------------------+--------------------+--------------------+
|    0|[16, 14, 10, 13, 12]|[0.03445398074735...|[3320m, x230, 8, ...|
|    1|[343, 166, 339, 1...|[0.00301512143024...|[12tbdcckk, 4290,...|
|    2|[184, 368, 86, 13...|[0.00288863819464...|[v7, burner, 4010...|
|    3|[37, 88, 51, 18, 44]|[0.02266808435478...|[i3, x130e, 11, 6...|
|    4|[23, 39, 60, 27, 24]|[0.03234836003291...|[computers, inch,...|
|    5|[47, 40, 182, 246...|[0.05826819160151...|[elitebook, hp, 2...|
|    6|[375, 84, 211, 20...|[0.00295990194561...|[5423, pc, touchs...|
|    7|[7, 278, 307, 245...|[0.00295240715391...|[thinkpad, gtx, b...|
|    8|[89, 75, 144, 95,...|[0.02066701435931...|[771, 17, mg7aa, ...|
|    9|[220, 237, 281, 3...|[0.00308689832019...|[20204g50mnii, 73...|
+-----+--------------------+--------------------+--------------------+

+----

In [300]:
blocking_df = blocking_df.withColumn('title_encoding', encode_sentence(f.coalesce(f.col('title'), f.lit(''))))\
.withColumn('blocking_keys',f.col('title_swRemoved_top_tokens'))
blocking_df.toPandas()

AnalysisException: cannot resolve '`title_swRemoved_top_tokens`' given input columns: [brand, brand_swRemoved, cpu_brand, cpu_brand_swRemoved, cpu_brand_swRemoved_RawFeatures, cpu_brand_swRemoved_features, cpu_brand_swRemoved_tfidf, cpu_frequency, cpu_model, cpu_type, dimensions, hdd_capacity, instance_id, ram_capacity, ram_frequency, ram_type, ssd_capacity, title, title_encoding, title_swRemoved, weight];
'Project [instance_id#48429, brand#48654, cpu_brand#48677, cpu_model#48692, cpu_type#48489, cpu_frequency#48504, ram_capacity#48519, ram_type#48534, ram_frequency#48549, hdd_capacity#48564, ssd_capacity#48579, weight#48594, dimensions#48609, title#48624, title_swRemoved#48765, brand_swRemoved#48928, cpu_brand_swRemoved#49099, cpu_brand_swRemoved_RawFeatures#49517, cpu_brand_swRemoved_features#49546, cpu_brand_swRemoved_tfidf#49572, title_encoding#49657, 'title_swRemoved_top_tokens AS blocking_keys#49679]
+- Project [instance_id#48429, brand#48654, cpu_brand#48677, cpu_model#48692, cp

In [ ]:
blocking_df.groupby('blocking_keys').count().show()